In [ ]:
%pip install -q --disable-pip-version-check \
    evaluate==0.4.0 \
    py7zr==0.20.4 \
    sentencepiece==0.1.99 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.4.0 \
    trl==0.7.2
%pip install -q    wandb bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.4 M

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
PROJECT = "FlanT5-Lora"
MODEL_NAME = 'google/flan-t5-base'
DATASET = "knkarthick/dialogsum"

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

# tqdm library makes the loops show a smart progress meter.
from tqdm import tqdm
tqdm.pandas()

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!ls

test.jsonl  train.jsonl  validation.jsonl


In [ ]:
import os
os.chdir('drive/MyDrive/nlp_proj_dataset')

In [ ]:
from datasets import load_dataset

# Define the paths to your files

def create_hf_dataset():
    data_files = {}
    data_files["train"] = 'train.jsonl'
    data_files["test"]  = 'test.jsonl'
    data_files["validation"] = 'validation.jsonl'

    # Load the datasets
    dataset = load_dataset('json', data_files=data_files)

    # Print the first example of the training dataset
    print(dataset)

    return dataset

# check the number of examples in each dataset

dataset = create_hf_dataset()

DatasetDict({
    train: Dataset({
        features: ['idx', 'inputs', 'target', 'inputs_tagged'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['idx', 'inputs', 'target', 'inputs_tagged'],
        num_rows: 150
    })
    validation: Dataset({
        features: ['idx', 'inputs', 'target', 'inputs_tagged'],
        num_rows: 50
    })
})


In [ ]:
dataset['train'].to_pandas()

,idx,inputs,target,inputs_tagged
0,0,SUBJECT: who and where to get cetirizine - D M...,Who manufactures cetirizine?,SUBJECT: who and where to get cetirizine - D M...
1,1,who makes bromocriptine i am wondering what co...,Who manufactures bromocriptine?,who makes <DIAGNOSTIC_PROCEDURE> bromocriptine...
2,2,SUBJECT: nulytely MESSAGE: Hello can you tell ...,"Who makes nulytely, and where can I buy it?",SUBJECT: nulytely MESSAGE: Hello can you tell ...
3,3,Williams' syndrome I would like to have my dau...,Where can I get genetic testing for william's ...,<DISEASE_DISORDER> williams' syndrome I would ...
4,4,ClinicalTrials.gov - Question - general inform...,Where can I get genetic testing for multiple m...,ClinicalTrials.gov - Question - general inform...
...,...,...,...,...
995,995,SUBJECT: after surgery of ear drum still same ...,What are the treatments for perforated eardrum?,SUBJECT: after <THERAPEUTIC_PROCEDURE> surgery...
996,996,SUBJECT: ClinicalTrials.gov - Question - speci...,What are the treatments for Glycogen storage d...,SUBJECT: <DIAGNOSTIC_PROCEDURE> clinicaltrials...
997,997,MESSAGE: I have numbness/tingling in my lower ...,Where can I find information and treatment for...,MESSAGE: I have numbness/tingling in my <BIOLO...
998,998,SUBJECT: sleep apnea MESSAGE: I was diagnosed ...,How long does swelling from sleep apnea take t...,SUBJECT: <DISEASE_DISORDER> sleep apnea MESSAG...


In [ ]:
model_name='google/flan-t5-base'
#model_name = 'google/flan-t5-small'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
def print_number_of_trainable_model_parameters(model, tag="original_model"):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    '''
    with wandb.init(project=PROJECT, job_type="log_parameters"):
      wandb.log({f'{tag}': {"trainable_model_params":trainable_model_params}})
      wandb.log({f'{tag}': {"all_model_params":all_model_params}})
      wandb.log({f'{tag}': {"percentage_of_trainable_model_parameters": 100 * trainable_model_params}} )
    '''

    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.0%


In [ ]:
lindex = [i for i in range(0,150)]
human_baseline_summaries = dataset['test'][0:150]['target']
#lindex = [i for i in range(0,10)]
all_outputs = []
for index in lindex:
  dialogue = dataset['test'][index]['inputs']
  summary = dataset['test'][index]['target']

  prompt = f"""
  Read through the whole context and summarize the medical question focusing on tags supplied within <> brackets.

  {dialogue}

  Summary:
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  output = tokenizer.decode(
      original_model.generate(
          inputs["input_ids"],
          max_new_tokens=200,
      )[0],
      skip_special_tokens=True
  )
  all_outputs.append(output)

  dash_line = ('-'.join('' for x in range(100)))
  print(dash_line)
  print(f'INPUT PROMPT:\n{prompt}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
  print(dash_line)
  print(f'MODEL GENERATION - ZERO SHOT WITH NER:\n{output}')
  #table.add_data(index,dialogue,prompt,summary,output)
print(all_outputs)
import os
try:
  os.mkdir("results_qualitative")
except:
  pass
f = open('results_qualitative/zero-shot_ner_flanbase.txt','w')
f.write(str(all_outputs))
f.close()

#f2 = open('results_qualitative/human_summaries_ner_flanbase.txt','w')
#f2.write(str(human_baseline_summaries))
#f2.close()


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

  Read through the whole context and summarize the medical question focusing on tags supplied within <> brackets.

  23 surgeries and counting......lower lip birthmark, have tried all options out the there and guess what still have it, continues to grow back.....any suggestions? Is there a cure coming in the next few years hopefully?

  Summary:
  
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
How can i get rid of a lower lip birthmark permanently?

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT WITH NER:
What is the best way to treat a lip birthmark?
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

  Read through the whole context and summarize the m

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
rogue_res = rouge.compute(
    predictions=all_outputs,
    references=human_baseline_summaries[0:len(all_outputs)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(rogue_res)

try:
  os.mkdir('quantitative_results')
except:
  pass


ORIGINAL MODEL:
{'rouge1': 0.23476766469346652, 'rouge2': 0.08726150706049832, 'rougeL': 0.2121594144419069, 'rougeLsum': 0.212036163433865}


In [ ]:
bleu = evaluate.load("bleu")

In [ ]:
bleu_res = bleu.compute(
    predictions=all_outputs,
    references=human_baseline_summaries[0:len(all_outputs)]
)

print('ORIGINAL MODEL:')
print(bleu_res)


ORIGINAL MODEL:
{'bleu': 0.048472998491384, 'precisions': [0.19594302977988778, 0.063682510383018, 0.03070827142149579, 0.0144076840981857], 'brevity_penalty': 1.0, 'length_ratio': 1.2973124300111982, 'translation_length': 2317, 'reference_length': 1786}


In [ ]:
!pip install bert_score

In [ ]:
from evaluate import load
bertscore = load("bertscore")
predictions = all_outputs
references = human_baseline_summaries[0:len(all_outputs)]
results = bertscore.compute(predictions=predictions, references=references, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
mean_precision = np.mean(np.array(results['precision']))
mean_recall = np.mean(np.array(results['recall']))
mean_f1 = np.mean(np.array(results['f1']))
print("Precision: " +str(mean_precision))
print("Recall: " +str(mean_recall))
print("F1-score: " +str(mean_f1))


Precision: 0.8783423185348511
Recall: 0.8752063250541687
F1-score: 0.8764081919193267


In [ ]:
f1 = open('quantitative_results/zero-shot_flanbase_ner.txt','w')
finalstr = ""
finalstr += "Rogue:\n"
finalstr += str(rogue_res)+"\n"
finalstr += "Bleu:\n"
finalstr += str(bleu_res)+"\n"
finalstr +="BertScore:\n"
finalstr +="Precision: "+str(mean_precision)+"\n"
finalstr +="Recall: "+str(mean_recall)+"\n"
finalstr +="F1-score: "+str(mean_f1)
f1.write(finalstr)
f1.close()